In [2]:
import os
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.cm import register_cmap
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, FactorAnalysis

import django
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits

from marslab.imgops.pltutils import attach_axis, despine
import matplotlib as mpl
from fit import fit_line, plot_mesh, correlation_matrix, Fit, numeric_columns


os.chdir('..')


os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()
from marslab.imgops.imgutils import normalize_range
import plotter.models
from plotter.spectrum_ops import filter_df_from_queryset

from marslab.compat.xcam import DERIVED_CAM_DICT

from multidex_utils import modeldict, model_metadata_df
pd.set_option('display.max_rows', 200)
# %matplotlib qt
# %matplotlib qt
plt.rcParams['font.size'] = '10'

tl;dr: 
1. I'm writing PCA features for MultiDEx. In almost all cases, almost all of the variance is loaded in the first and second principal components, and they are really close to just adding metrics that are (for ZCAM) 'average value of L6 through R0G' and 'average value of L4 through R6'.  and (for MCAM) 'overall ROI brightness' and 'average value of L2 through L0G'. There are some potentially-interesting things out in the tails, but I just wanted to note that vanilla PCA _could_ be questionably useful, and suggest that we should continue exploring different approaches to dimensionality reduction on these data.
2. I _think_ we're applying R* 'correctly' everywhere, but I also think that for whatever reason it is making the values more rather than less dependent on 'incidence angle' (which is to say solar elevation). More broadly, I suspect that more granular illumination geometry is a huge latent factor in a lot of these statistics.
3. Based on my exchanges with Tina about MCAM I think there are incorrect values in the lookup table I received for


the r-star correction definitely makes the effect of incidence angle on reflectance more rather than less pronounced. when applied, incidence angle becomes the single largest determinant of reflectance values rather than a ~20% determinant of reflectance values. this could of course result from shared latent
factors. for instance, our corpus is simply not that large and solar elevation varies scene-by-scene; if certain type of observations tend to be performed at certain times of day, for instance, or merely by
coincidence have been. and this does seem to be the case -- look at the relatively high correlation between lat, lon, sclk, and incidence angle, all of which should be basically random (or periodic in the case of sclk, but that wouldn't show up here).

it's worth noting, though, that even in the larger MCAM corpus, applying r-star seems to make the overall effect of incidence angle more pronunced, although not to the same degree.

It could 

looking at some of the values I got from Tina the other day, I think some of the mappings I have between MCAM ROI colors and feature types might be wrong. Was there a miscommunication about this at some point? If so, do you have a not-wrong version of this table?

there is one very large factor that is something like overall ROI brightness; then another that is something like "blue-greenness'. you can see this in the correlation matrices without any explicit dimensionality reduction -- L6-R0G is one cluster; L4-R6 is another (and L4-R6 has _really_ high internal correlation).

This means that a lot of the interesting action is out in the statistical tails -- this is something we know about these data in general, but it really underscores, I think, how important it might be to 

In [ ]:
plt.close('all')

In [3]:
corr_cmap = 'orange_teal'
instrument = 'ZCAM'
spec_model = getattr(
    plotter.models, 
    plotter.models.INSTRUMENT_MODEL_MAPPING[instrument]
)
filter_info = DERIVED_CAM_DICT[instrument]['filters']
filts = list(filter_info.keys())
narrowband = [filt for filt in filts if len(filt) == 2]
metadata_df = model_metadata_df(spec_model)

r_star = False
# scale_to = ('L1', 'R1')
norm_values = False
# scale_to = ('L0B', 'R0B')
# scale_to = ('L6', 'R6')
# scale_to=None
data_df = filter_df_from_queryset(
    spec_model.objects.all(), r_star=r_star, scale_to=scale_to
)
corpus = pd.concat([metadata_df, data_df], axis=1)
corpus['ltst'] = corpus['ltst'].map(s_from_midnight)
corpus['avg'] = corpus[filts].mean(axis=1)
explode_field = 'morphology'
# explode_field = None




# fields to do pca on
# pca_fields = numeric_columns(search)
pca_fields = filts
# pca_fields = [band + "_err" for band in narrowband]
# fields to compare with the PCs 
# corr_fields = ['incidence_angle', 'lat', 'lon', 'sclk']
corr_fields =  filts + ['avg', 'incidence_angle'] + list(exploded.columns)
# corr_fields += [band + "_err" for band in narrowband]

# search_fields = [('feature', 'rock')]
# search = search.loc[search['incidence_angle'] > -10]


TypeError: getattr(): attribute name must be string

In [ ]:
fig, ax = plt.subplots()

corrchart = ax.imshow(correlations, norm=offset, cmap='Spectral')

ax.set_yticks(np.arange(len(correlations.index)))
ax.set_yticklabels([ix[:4] for ix in correlations.index])
ax.set_xticks(np.arange(len(correlations.columns)))
ax.set_xticklabels([ix[:4] for ix in correlations.columns])
plt.title(
    "{} scaled to {} with R* {}"\
    .format(instrument, str(scale_to), str(r_star))
)
cax = attach_axis(ax, 'right', '10%')
plt.colorbar(corrchart, cax=cax)

In [ ]:
plt.imshow(corre)

In [ ]:
plt.close('all')

In [ ]:
def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean", sample_size=300,)
    ]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}")
    print(formatter_result.format(*results))

In [ ]:
data, labels = load_digits(return_X_y=True)
(n_samples, n_features), n_digits = data.shape, np.unique(labels).size

print(
    f"# digits: {n_digits}; # samples: {n_samples}; # features {n_features}"
)

print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

kmeans = KMeans(init="k-means++", n_clusters=n_digits, n_init=4,
                random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=data, labels=labels)

kmeans = KMeans(init="random", n_clusters=n_digits, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=data, labels=labels)

pca = PCA(n_components=n_digits).fit(data)
kmeans = KMeans(init=pca.components_, n_clusters=n_digits, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=data, labels=labels)

print(82 * '_')

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
data_df = filter_df_from_queryset(ZSpec.objects.all(), r_star=True)
data_df.drop(columns=[col for col in data_df.columns if "err" in col], inplace=True)

In [ ]:
data_df

In [ ]:
km = KMeans(n_clusters=10)
metadata_df = model_metadata_df(ZSpec, ["observation"])
metadata_df['km'] = km.fit_predict(data_df)
metadata_df[['feature', 'name', 'km']]